In [38]:
# As funções necessárias estão no arquivo functions.py devido ao fato das bibliotecas não funcionarem corretamente com o Jupyter Notebook
import os.path 
import pandas as pd
#import matplotlib.pyplot as plt
import multiprocessing as mp
import functions as f # biblioteca de funções
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import plotly.io as pio

# %matplotlib inline resolve o problema da geração de plots
#%matplotlib inline 
pd.set_option('display.max_columns', None) # mostra todas as colunas quando mostrando o dataframe
pd.set_option('display.max_rows', None) # mostra todas as linhas quando mostrando o dataframe


#data = pd.read_csv(('rolloff diferenca cumulativa M+2 SE -_ hip_2.csv'), index_col="data", parse_dates=True)
#data = data.rename(columns={'VWAP':'close'})

data0 = pd.read_csv(('rolloff diferenca cumulativa M+2 SE -_ hip_2.csv'), index_col="data", parse_dates=True)


In [39]:
#Definidas as estratégias: 
#- Simple Moving Average (SMA), Exponential Moving Average (EMA), Bollinger Bands (BBANDS)

# Chamando as funções "estrategias_gridsearch" para fazer o backtest da estratégia em busca dos melhores parâmetros (o range dos parametros a serem testados é editável no arquivo functions.py)
#f.estrategias_gridsearch(data0)

# Rodando as estratégias ótimas
# Calcula os indicadores de análise técnica com os parâmetros ótimos e gera gráficos das negociações
#f.estrategias_otimas(data0)

# Unifica os multiplos datasets de cada estratégia em 4 dataframes e filtra dados desnecessários 
#f.create_dataframes()

# Executa o machine learning
#f.execute_machine_learning(data0)

# Lê os dados unificados e divididos
dataframe_results = pd.read_csv(os.path.join('ML_data', 'scores_dataframe.csv'), index_col=1)
train_scores = pd.read_csv(os.path.join('ML_data', 'scores_train.csv'), index_col=1)
test_scores = pd.read_csv(os.path.join('ML_data', 'scores_test.csv'), index_col=1)
r2df = pd.read_csv(os.path.join('ML_data', 'round_2_dataframe.csv'), index_col=0, parse_dates=True)


linear_regression = 'linear_regression'
ridge_regression = 'ridge_regression'
logistic_regression = 'logistic_regression'
lgbm_regression = 'lgbm_regression'
lgbm_classifier = 'lgbm_classifier'
xgboost_regression = 'xgboost_regression'
xgboost_classifier = 'xgboost_classifier'
gnb = 'gnb'
random_forest= 'random_forest'
svc = 'svc'
linear_svc = 'linear_svc'

In [31]:

#r2df['position_logistic_regression'] = r2df['position_logistic_regression'].astype('bool')
#r2df['position_lgbm_classifier'] = r2df['position_lgbm_classifier'].astype('bool')
#r2df['position_xgboost_classifier'] = r2df['position_xgboost_classifier'].astype('bool')
#r2df['position_gnb'] = r2df['position_gnb'].astype('bool')
#r2df['position_random_forest'] = r2df['position_random_forest'].astype('bool')
#r2df['position_svc'] = r2df['position_svc'].astype('bool')
#r2df['position_linear_svc'] = r2df['position_linear_svc'].astype('bool')



In [32]:
def plot_regressions(df, model):
    
    # Dynamic column name based on the model
    position_column = f'position_{model}'
    return_column = f'return_{model}'

    # Convert to boolean and calculate entry/exit points
    df[position_column] = df[position_column] == 'True'
    df[f'entry_point_{model}'] = df[position_column] & ~df[position_column].shift(1).fillna(False)
    df[f'exit_point_{model}'] = ~df[position_column] & df[position_column].shift(1).fillna(False)

    # Calculate accumulated returns
    df[f'accumulated_return_{model}'] = (1 + df[return_column]).cumprod()

    # Create subplots with different heights
    # Adjust these values as needed for your desired plot sizes
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, 
                        subplot_titles=('Price and Trading Points', ),
                        row_heights=[0.85, 0.15])  # 70% height for first plot, 30% for second

    # First subplot for price and trading points
    fig.add_trace(go.Scatter(x=df.index, y=df['real_close'], mode='lines', name='Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df[df[f'entry_point_{model}']].index, y=df[df[f'entry_point_{model}']]['real_close'], mode='markers', name='Entry Points', marker=dict(color='green', size=10)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df[df[f'exit_point_{model}']].index, y=df[df[f'exit_point_{model}']]['real_close'], mode='markers', name='Exit Points', marker=dict(color='red', size=10)), row=1, col=1)

    # Second subplot for accumulated returns
    fig.add_trace(go.Scatter(x=df.index, y=df[f'accumulated_return_{model}'], mode='lines', name='Accumulated Return'), row=2, col=1)

    # Update layout
    fig.update_layout(title=f'{model} - Price and Accumulated Returns', height=600, width=800)
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_yaxes(title_text='Accumulated Return', row=2, col=1)

    # Show the figure
    fig.show()

    # Save the plot as a PNG
    file_name = f"{model}_plot.png"
    pio.write_image(fig, file_name)

    df.to_csv('dataset_completo.csv')

    print(f"Plot saved as {file_name}")
    return


In [52]:
def plot_classifier(df, model):
    
    # Dynamic column names based on the model
    position_column = f'position_{model}'
    return_column = f'return_{model}'

    # Handle NaN values and convert to boolean
    df[position_column] = df[position_column].fillna(False).astype(bool)

    # Calculate entry and exit points
    df[f'entry_point_{model}'] = df[position_column] & ~df[position_column].shift(1).fillna(False)
    df[f'exit_point_{model}'] = ~df[position_column] & df[position_column].shift(1).fillna(False)

    # Calculate accumulated returns, handle NaN values in return_column
    df[return_column] = df[return_column].fillna(0)  # Assuming 0 return for NaNs
    df[f'accumulated_return_{model}'] = (1 + df[return_column]).cumprod()

    # Create subplots with different heights
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                        subplot_titles=('Price and Trading Points', 'Accumulated Returns'),
                        row_heights=[0.85, 0.15])

    # First subplot for price and trading points
    fig.add_trace(go.Scatter(x=df.index, y=df['real_close'], mode='lines', name='Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df[df[f'entry_point_{model}']].index, y=df[df[f'entry_point_{model}']]['real_close'], mode='markers', name='Entry Points', marker=dict(color='green', size=10)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df[df[f'exit_point_{model}']].index, y=df[df[f'exit_point_{model}']]['real_close'], mode='markers', name='Exit Points', marker=dict(color='red', size=10)), row=1, col=1)

    # Second subplot for accumulated returns
    fig.add_trace(go.Scatter(x=df.index, y=df[f'accumulated_return_{model}'], mode='lines', name='Accumulated Return'), row=2, col=1)

    # Update layout
    fig.update_layout(title=f'{model} - Price and Accumulated Returns', height=600, width=800)
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_yaxes(title_text='Accumulated Return', row=2, col=1)

    # Show and save the figure
    fig.show()
    file_name = f"{model}_plot.png"
    pio.write_image(fig, file_name)

    # Save the modified DataFrame (optional)
    df.to_csv('dataset_completo.csv')

    print(f"Plot saved as {file_name}")
    return

# Example of how to call the function
# plot_classifier(df, 'lgbm_classifier')


In [22]:
r2df

,SMA_fast,SMA_slow,SMA_crossover,EMA_10,EMA_88,CrossOver_y,BBANDS_mid,BBANDS_top,BBANDS_bot,close,return,return_shifted,BBANDS_signal,real_close,ideal_signal_(d+1),position_linear_regression,return_linear_regression,position_ridge_regression,return_ridge_regression,position_logistic_regression,return_logistic_regression,position_lgbm_regression,return_lgbm_regression,position_lgbm_classifier,return_lgbm_classifier,position_xgboost_regression,return_xgboost_regression,position_xgboost_classifier,return_xgboost_classifier,position_gnb,return_gnb,position_random_forest,return_random_forest,position_svc,return_svc,position_linear_svc,return_linear_svc
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-16,246.906994,265.047060,0.0,248.195639,275.889851,0.0,5.517166,5.537101,5.496825,5.509012,-0.000036,0.001103,-1,246.906994,1,0,0.000000,0,0.000000,True,NaN,0,0.000000,True,NaN,0,0.000000,True,NaN,True,NaN,True,NaN,True,NaN,True,NaN
2022-10-17,247.179435,264.223640,0.0,248.010874,275.244673,0.0,5.515875,5.534277,5.497128,5.510115,0.001103,0.000312,-1,247.179435,1,True,0.000312,False,0.000000,False,0.0,True,0.000312,True,0.000312,True,0.000312,False,0.0,False,0.000000,True,0.000312,True,0.000312,False,0.0
2022-10-18,247.256663,263.423482,0.0,247.873745,274.615729,0.0,5.514713,5.531219,5.497930,5.510427,0.000312,0.001997,-1,247.256663,1,True,0.001997,False,0.000000,False,0.0,True,0.001997,True,0.001997,True,0.001997,False,0.0,False,0.000000,True,0.001997,True,0.001997,False,0.0
2022-10-19,247.750950,262.659219,0.0,247.851419,274.012026,0.0,5.513812,5.528023,5.499395,5.512424,0.001997,-0.001764,-1,247.750950,-1,True,-0.001764,False,-0.000000,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001764,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001764,False,-0.0
2022-10-20,247.314356,261.885412,0.0,247.753771,273.412078,0.0,5.512974,5.525465,5.500326,5.510660,-0.001764,0.000000,-1,247.314356,0,True,0.000000,False,0.000000,False,0.0,True,0.000000,True,0.000000,True,0.000000,False,0.0,False,0.000000,False,0.000000,True,0.000000,False,0.0
2022-10-21,247.314356,261.101508,0.0,247.673877,272.825613,0.0,5.512132,5.521956,5.502211,5.510660,0.000000,-0.000702,-1,247.314356,-1,False,-0.000000,False,-0.000000,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000702,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000702,False,-0.0
2022-10-22,247.140818,260.326046,0.0,247.576957,272.248426,0.0,5.511342,5.517970,5.504670,5.509958,-0.000702,-0.001052,-1,247.140818,-1,False,-0.000000,False,-0.000000,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001052,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001052,False,-0.0
2022-10-23,246.880907,259.549139,0.0,247.450403,271.678370,0.0,5.510783,5.515926,5.505614,5.508906,-0.001052,-0.000593,-1,246.880907,-1,False,-0.000000,False,-0.000000,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000593,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000593,False,-0.0
2022-10-24,246.734599,258.804062,0.0,247.320257,271.117836,0.0,5.510321,5.514590,5.506034,5.508313,-0.000593,-0.000351,-1,246.734599,-1,True,-0.000351,False,-0.000000,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000351,False,-0.0,False,-0.000000,False,-0.000000,True,-0.000351,False,-0.0


,SMA_fast,SMA_slow,SMA_crossover,EMA_10,EMA_88,CrossOver_y,BBANDS_mid,BBANDS_top,BBANDS_bot,close,return,return_shifted,BBANDS_signal,real_close,ideal_signal_(d+1),return_linear_regression,return_ridge_regression,return_logistic_regression,return_lgbm_regression,return_lgbm_classifier,return_xgboost_regression,position_xgboost_classifier,return_xgboost_classifier,return_gnb,return_random_forest,return_svc,return_linear_svc
count,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,368.000000,369.000000,369.000000,369.000000,368.000000,368.000000,367.0,368.000000,367.000000,368.000000,367.0,367.0,367.000000,367.000000,367.000000,367.0
mean,247.855622,248.485368,0.002710,247.824753,251.159024,0.002710,5.512577,5.520533,5.504475,5.512796,0.000026,0.000026,-0.943089,247.855622,0.027100,0.000047,-0.000008,0.0,0.000244,0.000261,0.000023,0.0,0.0,0.000035,0.000283,0.000023,0.0
std,2.485567,2.939367,0.156362,2.221756,5.774339,0.090249,0.008940,0.013023,0.012401,0.010027,0.002805,0.002809,0.274876,2.485567,0.629287,0.002545,0.000833,0.0,0.001937,0.001955,0.002808,0.0,0.0,0.001014,0.001758,0.002812,0.0
min,242.601946,245.328017,-1.000000,242.791007,246.398644,-1.000000,5.491422,5.491422,5.457755,5.491422,-0.020493,-0.020493,-1.000000,242.601946,-1.000000,-0.020493,-0.013356,-0.0,-0.015339,-0.015339,-0.020493,0.0,-0.0,-0.013356,-0.007121,-0.020493,-0.0
25%,246.386698,246.887611,0.000000,246.582246,247.995041,0.000000,5.507527,5.512454,5.498023,5.506902,0.000000,0.000000,-1.000000,246.386698,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
50%,247.750394,247.724640,0.000000,247.706918,249.111438,0.000000,5.512137,5.518401,5.507937,5.512422,0.000000,0.000000,-1.000000,247.750394,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
75%,249.559846,249.490020,0.000000,249.282957,250.599099,0.000000,5.518553,5.527321,5.513054,5.519699,0.000000,0.000000,-1.000000,249.559846,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
max,255.629702,265.047060,1.000000,252.959634,275.889851,1.000000,5.533440,5.565284,5.525412,5.543730,0.023033,0.023033,1.000000,255.629702,1.000000,0.023033,0.008658,-0.0,0.023033,0.023033,0.023033,0.0,-0.0,0.010244,0.023033,0.023033,-0.0


In [33]:
plot_regressions(r2df, linear_regression), #ok
plot_regressions(r2df, ridge_regression), #ok
plot_regressions(r2df, lgbm_regression), #ok
plot_regressions(r2df, xgboost_regression), #ok

Plot saved as linear_regression_plot.png


Plot saved as ridge_regression_plot.png


Plot saved as logistic_regression_plot.png


Plot saved as lgbm_regression_plot.png


Plot saved as lgbm_classifier_plot.png


...

Plot saved as xgboost_classifier_plot.png


Plot saved as gnb_plot.png


Plot saved as random_forest_plot.png


Plot saved as svc_plot.png


Plot saved as linear_svc_plot.png


In [53]:
plot_classifier(r2df, logistic_regression), 
plot_classifier(r2df, lgbm_classifier),
plot_classifier(r2df, xgboost_classifier),
plot_classifier(r2df, gnb),
plot_classifier(r2df, random_forest),
plot_classifier(r2df, svc),
plot_classifier(r2df, linear_svc)

Plot saved as logistic_regression_plot.png


Plot saved as lgbm_classifier_plot.png


Plot saved as xgboost_classifier_plot.png


Plot saved as gnb_plot.png


Plot saved as random_forest_plot.png


Plot saved as svc_plot.png


Plot saved as linear_svc_plot.png


In [40]:
df = pd.read_csv('dataset_completo.csv', index_col=0, parse_dates=True)


In [43]:
r2df.head()

,SMA_fast,SMA_slow,SMA_crossover,EMA_10,EMA_88,CrossOver_y,BBANDS_mid,BBANDS_top,BBANDS_bot,close,return,return_shifted,BBANDS_signal,real_close,ideal_signal_(d+1),position_linear_regression,return_linear_regression,position_ridge_regression,return_ridge_regression,position_logistic_regression,return_logistic_regression,position_lgbm_regression,return_lgbm_regression,position_lgbm_classifier,return_lgbm_classifier,position_xgboost_regression,return_xgboost_regression,position_xgboost_classifier,return_xgboost_classifier,position_gnb,return_gnb,position_random_forest,return_random_forest,position_svc,return_svc,position_linear_svc,return_linear_svc
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-16,246.906994,265.047060,0.0,248.195639,275.889851,0.0,5.517166,5.537101,5.496825,5.509012,-0.000036,0.001103,-1,246.906994,1,0,0.000000,0,0.0,NaN,NaN,0,0.000000,NaN,NaN,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-17,247.179435,264.223640,0.0,248.010874,275.244673,0.0,5.515875,5.534277,5.497128,5.510115,0.001103,0.000312,-1,247.179435,1,True,0.000312,False,0.0,False,0.0,True,0.000312,True,0.000312,True,0.000312,0.0,0.0,False,0.0,True,0.000312,True,0.000312,False,0.0
2022-10-18,247.256663,263.423482,0.0,247.873745,274.615729,0.0,5.514713,5.531219,5.497930,5.510427,0.000312,0.001997,-1,247.256663,1,True,0.001997,False,0.0,False,0.0,True,0.001997,True,0.001997,True,0.001997,0.0,0.0,False,0.0,True,0.001997,True,0.001997,False,0.0
2022-10-19,247.750950,262.659219,0.0,247.851419,274.012026,0.0,5.513812,5.528023,5.499395,5.512424,0.001997,-0.001764,-1,247.750950,-1,True,-0.001764,False,-0.0,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001764,0.0,-0.0,False,-0.0,False,-0.000000,True,-0.001764,False,-0.0
2022-10-20,247.314356,261.885412,0.0,247.753771,273.412078,0.0,5.512974,5.525465,5.500326,5.510660,-0.001764,0.000000,-1,247.314356,0,True,0.000000,False,0.0,False,0.0,True,0.000000,True,0.000000,True,0.000000,0.0,0.0,False,0.0,False,0.000000,True,0.000000,False,0.0


In [44]:
df.head()

,SMA_fast,SMA_slow,SMA_crossover,EMA_10,EMA_88,CrossOver_y,BBANDS_mid,BBANDS_top,BBANDS_bot,close,return,return_shifted,BBANDS_signal,real_close,ideal_signal_(d+1),position_linear_regression,return_linear_regression,position_ridge_regression,return_ridge_regression,position_logistic_regression,return_logistic_regression,position_lgbm_regression,return_lgbm_regression,position_lgbm_classifier,return_lgbm_classifier,position_xgboost_regression,return_xgboost_regression,position_xgboost_classifier,return_xgboost_classifier,position_gnb,return_gnb,position_random_forest,return_random_forest,position_svc,return_svc,position_linear_svc,return_linear_svc,entry_point_linear_regression,exit_point_linear_regression,accumulated_return_linear_regression,entry_point_ridge_regression,exit_point_ridge_regression,accumulated_return_ridge_regression,entry_point_logistic_regression,exit_point_logistic_regression,accumulated_return_logistic_regression,entry_point_lgbm_regression,exit_point_lgbm_regression,accumulated_return_lgbm_regression,entry_point_lgbm_classifier,exit_point_lgbm_classifier,accumulated_return_lgbm_classifier,entry_point_xgboost_regression,exit_point_xgboost_regression,accumulated_return_xgboost_regression,entry_point_xgboost_classifier,exit_point_xgboost_classifier,accumulated_return_xgboost_classifier,entry_point_gnb,exit_point_gnb,accumulated_return_gnb,entry_point_random_forest,exit_point_random_forest,accumulated_return_random_forest,entry_point_svc,exit_point_svc,accumulated_return_svc,entry_point_linear_svc,exit_point_linear_svc,accumulated_return_linear_svc
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-16,246.906994,265.047060,0.0,248.195639,275.889851,0.0,5.517166,5.537101,5.496825,5.509012,-0.000036,0.001103,-1,246.906994,1,False,0.000000,False,0.0,False,NaN,False,0.000000,False,NaN,False,0.000000,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,False,1.000000,False,False,1.0,False,False,NaN,False,False,1.000000,False,False,NaN,False,False,1.000000,False,False,NaN,False,False,NaN,False,False,NaN,False,False,NaN,False,False,NaN
2022-10-17,247.179435,264.223640,0.0,248.010874,275.244673,0.0,5.515875,5.534277,5.497128,5.510115,0.001103,0.000312,-1,247.179435,1,True,0.000312,False,0.0,False,0.0,True,0.000312,False,0.000312,True,0.000312,False,0.0,False,0.0,False,0.000312,False,0.000312,False,0.0,True,False,1.000312,False,False,1.0,False,False,1.0,True,False,1.000312,False,False,1.000312,True,False,1.000312,False,False,1.0,False,False,1.0,False,False,1.000312,False,False,1.000312,False,False,1.0
2022-10-18,247.256663,263.423482,0.0,247.873745,274.615729,0.0,5.514713,5.531219,5.497930,5.510427,0.000312,0.001997,-1,247.256663,1,True,0.001997,False,0.0,False,0.0,True,0.001997,False,0.001997,True,0.001997,False,0.0,False,0.0,False,0.001997,False,0.001997,False,0.0,False,False,1.002310,False,False,1.0,False,False,1.0,False,False,1.002310,False,False,1.002310,False,False,1.002310,False,False,1.0,False,False,1.0,False,False,1.002310,False,False,1.002310,False,False,1.0
2022-10-19,247.750950,262.659219,0.0,247.851419,274.012026,0.0,5.513812,5.528023,5.499395,5.512424,0.001997,-0.001764,-1,247.750950,-1,True,-0.001764,False,-0.0,False,-0.0,False,-0.000000,False,-0.000000,True,-0.001764,False,-0.0,False,-0.0,False,-0.000000,False,-0.001764,False,-0.0,False,False,1.000542,False,False,1.0,False,False,1.0,False,True,1.002310,False,False,1.002310,False,False,1.000542,False,False,1.0,False,False,1.0,False,False,1.002310,False,False,1.000542,False,False,1.0
2022-10-20,247.314356,261.885412,0.0,247.753771,273.412078,0.0,5.512974,5.525465,5.500326,5.510660,-0.001764,0.000000,-1,247.314356,0,True,0.000000,False,0.0,False,0.0,True,0.000000,False,0.000000,True,0.000000,False,0.0,False,0.0,False,0.000000,False,0.000000,False,0.0,False,False,1.000542,False,False,1.0,False,False,1.0,True,False,1.002310,False,False,1.002310,False,False,1.000542,False,False,1.0,False,False,1.0,False,False,1.002310,False,Fa